In [ ]:
%pip install ultralytics

import torch
from ultralytics import YOLO

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


In [ ]:
# Path to your data.yaml
data_path = '../data/data.yaml'

# Show the data.yaml content
with open(data_path, 'r') as f:
    print(f.read())


In [ ]:
model = YOLO('/Users/maximilian/Documents/Macbook Air 13/Studium/projects/wardrobe_detection_yolo/outputs/models/yolov8n_finetuned_v2.pt')
results = model.train(
    data='../data/data.yaml',
    epochs=100,
    patience=10,
    imgsz=640,
    batch=8,
    name='wardrobe_yolov8n_macos',
    project='../outputs'
)

# Run validation on the best model
metrics = model.val()
print(metrics)


In [ ]:
import cv2
from ultralytics import YOLO

# Load your trained model
model = YOLO('../outputs/models/yolov8n_finetuned.pt')

# Correct class label mapping
correct_names = [
    'adidas_samba',       # 0
    'adidas_spezial',     # 1
    'nike_killshot',      # 2
    'pants_dark',         # 3
    'pants_light',        # 4
    'shirt_lightblue',    # 5
    'shirt_lightlinen',   # 6
    'tshirt_dark',        # 7
    'tshirt_white'        # 8
]

# Open webcam
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Run YOLO prediction (suppress internal display)
    results = model.predict(frame, show=False)

    # Manually assign correct class names
    results[0].names = correct_names

    # Show prediction with correct labels
    annotated_frame = results[0].plot()

    # Display in OpenCV window
    cv2.imshow("YOLOv8 Inference", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import time
from ultralytics import YOLO

# Load your trained model
model = YOLO('../outputs/models/yolov8n_finetuned.pt')

# Correct class label mapping
correct_names = [
    'adidas_samba',
    'adidas_spezial',
    'nike_killshot',
    'pants_dark',
    'pants_light',
    'shirt_lightblue',
    'shirt_lightlinen',
    'tshirt_dark',
    'tshirt_white'
]

# Open webcam
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Initialize FPS measurement
prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    start_time = time.time()

    # Run YOLO prediction
    results = model.predict(frame, show=False)
    results[0].names = correct_names
    annotated_frame = results[0].plot()

    # FPS calculation
    end_time = time.time()
    fps = 1.0 / (end_time - start_time)

    # Overlay FPS on frame
    cv2.putText(
        annotated_frame,
        f"FPS: {fps:.2f}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    # Display
    cv2.imshow("YOLOv8 Inference", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()